In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# Data paths
base_path = r'D:\fake product\Data'
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')

# Enhanced preprocessing with CLAHE
def preprocess_images(dataset_path, image_size=(224, 224)):
    images, labels = [], []
    classes = sorted(os.listdir(dataset_path))
    class_map = {label: idx for idx, label in enumerate(classes)}
    
    for label in classes:
        label_path = os.path.join(dataset_path, label)
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                # Apply CLAHE for contrast enhancement
                lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
                l, a, b = cv2.split(lab)
                clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
                l = clahe.apply(l)
                lab = cv2.merge((l, a, b))
                img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
                
                img = cv2.resize(img, image_size)
                images.append(img)
                labels.append(class_map[label])
    
    return np.array(images) / 255.0, np.array(labels), class_map

# Load Data
X_train, y_train, class_map = preprocess_images(train_path, (224, 224))
X_test, y_test, _ = preprocess_images(test_path, (224, 224))

# Split Data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, stratify=y_train, random_state=42)

# Convert labels to categorical
num_classes = len(class_map)
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

# Class Weights for Balanced Training
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Augment Training Data
train_datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

train_generator = train_datagen.flow(X_train, y_train_cat, batch_size=32)

# Hybrid Model: EfficientNetB0 + CNN
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Add Custom CNN Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.0005))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(128, activation='relu', kernel_regularizer=l2(0.0005))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

output = Dense(num_classes, activation='softmax')(x)

# Build Model
model = Model(inputs=base_model.input, outputs=output)

# Compile Model
model.compile(optimizer=AdamW(learning_rate=0.0003), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for Faster Convergence
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=2, verbose=1)
]

# Train Model (Reduced Epochs)
history = model.fit(
    train_generator, 
    validation_data=(X_val, y_val_cat),
    epochs=20, 
    batch_size=32, 
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# Fine-tune by Unfreezing EfficientNet Layers
base_model.trainable = True
model.compile(optimizer=AdamW(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue Training (5 More Epochs for Fine-tuning)
history_finetune = model.fit(
    train_generator, 
    validation_data=(X_val, y_val_cat),
    epochs=5, 
    batch_size=32, 
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# Evaluate Model on Test Set
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {test_acc * 100:.2f}% | Test Loss: {test_loss:.4f}")

# Save Model
model.save('hybrid_model_fake_product.h5')


c:\Users\Sri_S\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 169s 932ms/step - accuracy: 0.1197 - loss: 3.2689 - val_accuracy: 0.1563 - val_loss: 2.2910 - learning_rate: 3.0000e-04
Epoch 2/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 148s 915ms/step - accuracy: 0.1216 - loss: 2.9138 - val_accuracy: 0.0820 - val_loss: 2.5289 - learning_rate: 3.0000e-04
Epoch 3/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 823ms/step - accuracy: 0.1290 - loss: 2.8199
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
162/162 ━━━━━━━━━━━━━━━━━━━━ 149s 917ms/step - accuracy: 0.1290 - loss: 2.8197 - val_accuracy: 0.0820 - val_loss: 2.5034 - learning_rate: 3.0000e-04
Epoch 4/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 136s 839ms/step - accuracy: 0.1317 - loss: 2.7152 - val_accuracy: 0.0820 - val_loss: 2.3589 - learning_rate: 1.5000e-04
Epoch 5/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 758ms/step - accuracy: 0.1157 - loss: 2.7070
Epoch 5: ReduceLROnPlateau reducing learning rate to 7.500000356230885e-05.
162/162 ━━━━━━━━━━━━━━━━━━━━ 139s 855ms/ste

Test Accuracy: 18.46% | Test Loss: 2.3706
